## Using OpenStreetMap as a Training Dataset

In this notebook we will access and explore OSM data around Austin, TX.  OSM categorizes it's data according to the Nominatim taxonomy.  This makes for an interesting source of training data to feed to models.  Among others, the terrapattern project has used this to good effect against a ResNet-34 neural net.

First let's do a little bit of setup, and establish an area of interest to explore.

In [59]:
from gbdxtools import Interface, TmsImage
MAPS_API_KEY = "pk.eyJ1IjoiZGlnaXRhbGdsb2JlIiwiYSI6ImNqMXkyZXZsODAwYWszMmsyM3lvZHBzMWsifQ.EYqlvq6QWczWsvrEDDTf7g"
from shapely.geometry import box, shape, mapping
from shapely import ops
import proj
from rtree.index import Index as SpatialIndex

gbdx = Interface()
aoi = box(-97.803125,30.230669,-97.667427,30.306355) # Austin, TX
boundary = 


We'll also setup a TmsImage instance to use later for visualization

In [61]:
tms = TmsImage(zoom=18)

[aoi]

In [63]:
tms.shape

(3, 67106304, 67108864)

Next we'll query VectorServices for OSM data in the region of interest.  We're going to ask for up to a million records back.

In [26]:
osm_data = gbdx.vectors.query(aoi.wkt, query="item_type:* AND ingest_source:OSM AND geom_type:Polygon", 
                              index="vector-osm-*", count=1E6)
print("{} features found in search area".format(len(osm_data)))

64665 features found in search area


Let's take a look at the original OpenStreetMap attributes of one feature

In [27]:
osm_data[0]["properties"]["attributes"]

{u'_osm_changeset': u'35577335',
 u'_osm_copyright': u'\xa9 OpenStreetMap contributors',
 u'_osm_license': u'http://www.opendatacommons.org/licenses/odbl',
 u'_osm_user_id': u'3369502',
 u'_osm_user_name': u'patisilva_atxbuildings',
 u'_osm_version': u'1',
 u'building': u'yes',
 u'height': u'5.9'}

It looks like this feature is a building and that, in general, feature attributes are stored in non-underscored fields of the attributes structure.  Let's reject the `height` key and a few other types of keys we know about.  We'll explore further using Python's Counter class.  Note that the reject condition (and additional pieces were developed by iteratively examining the results observed in the counter class).  There's no guarantee we've gotten it perfectly right and the experience around this is quite bad.

In [50]:
from collections import Counter
rejected_keys = set(["height", "import"])
reject_condition = lambda t: (t.startswith("_") 
                              or t.startswith("addr:")
                              or t.startswith("coa:")
                              or t.startswith("source")
                              or t.startswith("tiger:")
                              or t in rejected_keys)

osm_types = Counter([t.split(":")[0] for rec in osm_data for t,v in rec["properties"]["attributes"].iteritems() 
                     if not reject_condition(t) and v == "yes"])
osm_types.most_common(100)

[(u'building', 62401),
 (u'bench', 111),
 (u'area', 55),
 (u'access', 35),
 (u'lit', 31),
 (u'wheelchair', 29),
 (u'oneway', 12),
 (u'capacity', 7),
 (u'climbing', 6),
 (u'supervised', 5),
 (u'garage', 4),
 (u'toilets', 3),
 (u'shop', 2),
 (u'fee', 2),
 (u'fuel', 2),
 (u'historic', 2),
 (u'atm', 2),
 (u'takeaway', 2),
 (u'drive_through', 1),
 (u'bicycle', 1),
 (u'health_specialty', 1),
 (u'heritage', 1),
 (u'covered', 1),
 (u'emergency', 1),
 (u'foot', 1),
 (u'outdoor_seating', 1),
 (u'construction', 1)]

There are a variety of feature types here that could be useful in characterizing an area.  It is rather unfortunate that most are generically buildings rather than something more specific, but let's pretend and move on.

To view this dataset as training data, we want to create two helper functions.  One that takes a collection of features and converts them to a consistent feature vector (based on the categories above) and a second that quickly looks up features in a particular geographic region.

For the first function:

In [56]:
# setup a categorical basis with consistent order
basis = [k for k,v in osm_types.most_common(100)]

# use the identical counter technique and produce a vector where each element is 1 or 0
# depending on whether a feature type exists in the feature set.
def basis_vector(features):
    stats = Counter([t.split(":")[0] for rec in features for t,v in rec["properties"]["attributes"].iteritems() 
                     if not reject_condition(t) and v == "yes"])
    return [int(k in stats) for k in basis]

For the second function we need to create a client-side spatial index to look up features because querying VectorServices during training will be too slow.  We'll call the first helper function within the second so that it can be used directly by a training process.

In [58]:
osm_index = SpatialIndex()
for idx, rec in enumerate(osm_data):
    osm_index.add(idx, shape(rec["geometry"]).bounds, rec)

def lookup_basis_vector(g):
    features = [rec.object for rec in osm_index.intersection(g.bounds, objects=True)]
    return basis_vector(features)

In [10]:
# TODO: lookup from model
row_chunk = 301
col_chunk = 301

# NOTE: Dropping far boundary, should fix
row_lims = xrange(row_chunk, tms_region.shape[1], row_chunk)
col_lims = xrange(col_chunk, tms_region.shape[2], col_chunk)

training_data = []
for maxy, maxx in tqdm(cartesian_prod(row_lims, col_lims),
                       total=len(row_lims)*len(col_lims)):
    region = tms_region[:, (maxy-row_chunk):maxy, (maxx-col_chunk):maxx]
    training_data.append((region, lookup_basis_vector(region)))